# Tracing in OpenAI Agents SDK

Tracing helps you monitor, debug, and understand what happens during an agent’s execution — including tool calls, handoffs, reasoning steps, and model responses.

It provides a detailed log of each step the agent takes while processing a request. With tracing, you can see:

- How the model interpreted your prompt

- Which tools were called (and with what inputs/outputs)

- When handoffs occurred between agents

- Any errors or intermediate reasoning

Why It’s Useful

- Helps you debug agent behavior (e.g., why it didn’t call a tool).

- Makes it easier to optimize workflows and identify bottlenecks.

- Essential for observability when building complex multi-agent systems.

In short:

> Tracing in the Agents SDK provides deep visibility into your agents’ decisions, making it easier to debug, evaluate, and improve their behavior.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    # Prompt the user to enter the API key if not set
    import getpass
    api_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")
    if not api_key:
        raise ValueError("OPENAI_API_KEY variable is not set in the environment variables or provided by user.")

In [2]:
from agents import Agent, Runner, trace

joke_agent = Agent(
    name = "Joke Agent",
    instructions="You are a famous standup comedian, you will tell a single joke on the given topic"
)

topic = "AI"

language_agent = Agent(
    name="Language Agent",
    instructions="You are language expert. You are given a joke you need to rewrite it in a diffrent language"
)
with trace("Joke Generation and Translation"):
    joke_result = await Runner.run(joke_agent, topic)
    translated_result = await Runner.run(language_agent, f"Translate this joke into arabic {joke_result.final_output}")
    print(f"Orignal Joke:\n{joke_result.final_output}")
    print(f"Translated Joke:\n{translated_result.final_output}")


Orignal Joke:
Why did the AI go to therapy?  
Because it had too many unresolved issues in its cache!
Translated Joke:
لماذا ذهب الذكاء الاصطناعي إلى العلاج النفسي؟  
لأنه كان لديه الكثير من المشاكل غير المحلولة في ذاكرته المؤقتة!


In [3]:
from agents import Agent, Tool, Runner, RunContextWrapper, handoff, function_tool
from pydantic import BaseModel

class ManagerEscalation(BaseModel):
    issue: str # The issue that needs managerial attention
    reason: str # The reason why the issue is being escalated

@function_tool
def create_ticket(issue: str):
    """
    Create a ticket in the system for an issue to be resolved.
    Returns a confirmation message with the generated ticket ID.
    """
    print(f"Ticket created for issue: {issue}")
    return "Ticket created. Ticket ID: 12345"

def on_manager_handoff(context: RunContextWrapper, input: ManagerEscalation):
    print("Escalating to Manager Agent: ", input.issue)
    print("Reason for escalation: ", input.reason)

manager_agent = Agent(
    name="Manager Agent",
    handoff_description="Handles escalated issues that required managarial attention.",
    instructions="""
    You handle escalated issues that initial customer service agents couldn't resolve.
    You will recive the issue and the reason for escalation. If the issue can not be resolved immidiately for
    the customer, create a ticket in the system and inform the customer.
    """, 
    tools=[create_ticket]
)


customer_service_agent = Agent(
    name="Customer Service Agent",
    instructions=""""
    You are a customer service agent. You will assist customers with general inquirries and basic troubleshooting.
    If the issue can not be resolve, escalate it to the Manager along with the reason why you can not fix the issue yourself.
    """,
    handoffs=[handoff(
        manager_agent,
        on_handoff=on_manager_handoff,
        input_type=ManagerEscalation
        )
    ]
)

with trace("Customer Service Interaction"):
    result = await Runner.run(customer_service_agent, "I want refund, but your system won't let me process it. Your website is just blank for me.")
    print(result.final_output)

Escalating to Manager Agent:  Customer is unable to process a refund because the website displays a blank page and does not allow them to complete the refund process.
Reason for escalation:  Technical issue with the website is preventing the customer from accessing the refund feature. Needs escalation to manager for manual assistance or further troubleshooting.
Ticket created for issue: Customer is unable to process a refund because the website displays a blank page. Technical issue is preventing access to the refund process.
I'm sorry you're experiencing issues with our website and are unable to process your refund. I've created a support ticket for your case so our technical team can investigate and resolve the problem.

Ticket ID: 12345

You will be contacted shortly with an update or resolution. If you have any other questions in the meantime, please let me know. Thank you for your patience.
